In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import json
import pickle

### 1.Problem statement

to predict charges of medical insurance 

### 2. Data gathering

In [2]:
df = pd.read_csv("medical_insurance.csv")
df.head()

age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520

### 3. EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
df.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [5]:
df.describe()

age          bmi     children       charges
count  1338.000000  1338.000000  1338.000000   1338.000000
mean     39.207025    30.663397     1.094918  13270.422265
std      14.049960     6.098187     1.205493  12110.011237
min      18.000000    15.960000     0.000000   1121.873900
25%      27.000000    26.296250     0.000000   4740.287150
50%      39.000000    30.400000     1.000000   9382.033000
75%      51.000000    34.693750     2.000000  16639.912515
max      64.000000    53.130000     5.000000  63770.428010

### 4. Feature Engineering

#### 4.1 sex

In [6]:
df['sex'].value_counts().to_dict()

{'male': 676, 'female': 662}

In [7]:
df["sex"].replace({'male': 1, 'female': 0},inplace=True)

#### 4.2 smoker

In [8]:
df['smoker'].value_counts().to_dict()

{'no': 1064, 'yes': 274}

In [9]:
df["smoker"].replace({'no': 0, 'yes': 1},inplace=True)

#### 4.3 region

In [10]:
df=pd.get_dummies(df,columns=["region"])

In [11]:
df

age  sex     bmi  children  smoker      charges  region_northeast  \
0      19    0  27.900         0       1  16884.92400                 0   
1      18    1  33.770         1       0   1725.55230                 0   
2      28    1  33.000         3       0   4449.46200                 0   
3      33    1  22.705         0       0  21984.47061                 0   
4      32    1  28.880         0       0   3866.85520                 0   
...   ...  ...     ...       ...     ...          ...               ...   
1333   50    1  30.970         3       0  10600.54830                 0   
1334   18    0  31.920         0       0   2205.98080                 1   
1335   18    0  36.850         0       0   1629.83350                 0   
1336   21    0  25.800         0       0   2007.94500                 0   
1337   61    0  29.070         0       1  29141.36030                 0   

      region_northwest  region_southeast  region_southwest  
0                    0                 0                 1  
1                    0                 1                 0  
2                    0                 1                 0  
3                    1                 0                 0  
4                    1                 0                 0  
...                ...               ...               ...  
1333                 1                 0                 0  
1334                 0                 0                 0  
1335                 0                 1                 0  
1336                 0                 0                 1  
1337                 1                 0                 0  

[1338 rows x 10 columns]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   int64  
 1   sex               1338 non-null   int64  
 2   bmi               1338 non-null   float64
 3   children          1338 non-null   int64  
 4   smoker            1338 non-null   int64  
 5   charges           1338 non-null   float64
 6   region_northeast  1338 non-null   uint8  
 7   region_northwest  1338 non-null   uint8  
 8   region_southeast  1338 non-null   uint8  
 9   region_southwest  1338 non-null   uint8  
dtypes: float64(2), int64(4), uint8(4)
memory usage: 68.1 KB


In [13]:
#df["charges"]=df["charges"].apply(np.sqrt)

### 5. Feature selection

### 6. Model Training

In [13]:
x=df.drop("charges",axis=1)
y=df["charges"]

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=45)

In [15]:
lr=LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

### 7. model Evaluation

In [16]:
#train data evaluation
yp_train=lr.predict(x_train)
mse=mean_squared_error(y_train,yp_train)
print("MSE",mse)
mae=mean_absolute_error(y_train,yp_train)
print("MAE",mae)
rmse=np.sqrt(mse)
print("RMSE",rmse)
r2=r2_score(y_train,yp_train)
print("R2",r2)

MSE 36112407.14310906
MAE 4110.367959255671
RMSE 6009.359961186305
R2 0.7605646905517846


In [17]:
#test data evalution
yp_test=lr.predict(x_test)
mse=mean_squared_error(y_test,yp_test)
print("MSE",mse)
mae=mean_absolute_error(y_test,yp_test)
print("MAE",mae)
rmse=np.sqrt(mse)
print("RMSE",rmse)
r2=r2_score(y_test,yp_test)
print("R2",r2)

MSE 38368288.57279197
MAE 4435.096155453824
RMSE 6194.214120676809
R2 0.7035312341521324


### write a code for single user

In [18]:
### df.iloc[0,:]
age                =  19.
sex                = "male"
bmi                =  30
children           =   1
smoker             =  "no"
region             =  "southwest"

#charges            =  ?


In [19]:
prj_data={"sex":{'male': 1, 'female': 0},
         "smoker":{'no': 0, 'yes': 1},
         "columns":list(x.columns)}


In [20]:
column_names=x.columns
array=np.zeros(x.shape[1])
array [0]=age
array [1]=prj_data["sex"][sex]
array [2]=bmi
array [3]=children
array [4]=prj_data["smoker"][smoker]
region="region_"+region
region_index=prj_data["columns"].index(region)
array[region_index]=1
array

array([19.,  1., 30.,  1.,  0.,  0.,  0.,  0.,  1.])

In [21]:
np.around(lr.predict([array])[0],2)

C:\Users\dell\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


2852.16

### make files for API making

In [22]:
with open ("project_data.json", "w" ) as f:
    json.dump(prj_data,f)

In [23]:
with open ("Liner_model.pkl","wb") as f :
    pickle.dump(lr,f)